<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#class-Axes" data-toc-modified-id="class-Axes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>class <code>Axes</code></a></span><ul class="toc-item"><li><span><a href="#Tests" data-toc-modified-id="Tests-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Tests</a></span></li></ul></li><li><span><a href="#class-Grid" data-toc-modified-id="class-Grid-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>class <code>Grid</code></a></span><ul class="toc-item"><li><span><a href="#Tests" data-toc-modified-id="Tests-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Tests</a></span></li></ul></li><li><span><a href="#class-Cloud" data-toc-modified-id="class-Cloud-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>class <code>Cloud</code></a></span></li><li><span><a href="#Next?" data-toc-modified-id="Next?-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Next?</a></span></li><li><span><a href="#MODULE-ptcloud" data-toc-modified-id="MODULE-ptcloud-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>MODULE <code>ptcloud</code></a></span></li><li><span><a href="#Execute-ptcloud.py" data-toc-modified-id="Execute-ptcloud.py-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Execute <code>ptcloud.py</code></a></span></li><li><span><a href="#Import-`ptcloud.py" data-toc-modified-id="Import-`ptcloud.py-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Import `ptcloud.py</a></span></li><li><span><a href="#Dev-ptcloud.py" data-toc-modified-id="Dev-ptcloud.py-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Dev <code>ptcloud.py</code></a></span></li></ul></div>

## class `Axes`

In [ ]:
import sys     
import decimal
import numpy as np

# disallow scientific notation when printing
np.set_printoptions(suppress=True)


def _get_precision(x: float):
    return abs(decimal.Decimal(str(x)).as_tuple().exponent)


def _safe_arange(start, stop, step):
    return step * np.arange(start / step, stop / step)


def _get_axis_mids(res: float, min: float, max: float):
    """Get centers of cells on an input axis defined by its min, max, res.
    
    This necessarily convoluted routine ensures that all discrete points in a 
    point cloud will all fall inside the returned axis (cell centers) with
    the specified resolution.
    
    :param min: The minimum of all point positions along this axis.
    :type min: float.
    :param max: The maximum of all point positions along this axis.
    :type max: float.
    :param res: The resolution of the desired axis (centers) array.
    :type res: float.
    :returns:  numpy.array.
    
    """
    midpoint       = (max + min)*0.5
    sequence_left  = _safe_arange(midpoint - res, min - res, -res)
    sequence_right = _safe_arange(midpoint, max + res, res)
    midpoints      = np.concatenate([np.sort(sequence_left), sequence_right])
    return midpoints


class Axis:
    """The arrays and attributes that fully describe a cartesian axis.
    
    .. note::
        All attributes WITHOUT underscores describe axis properties.
        All attributes WITH underscores describe cell properties.    
    
    """
    
    def __init__(self, name: str, res: float, min: float, max: float):
        """Initializes an `Axis` class object.
        
        :param name: The name of this axis. Best to use one of [x, y, z].
        :type name: str.       
        :param min: The minimum of all point positions along this axis.
        :type min: float.
        :param max: The maximum of all point positions along this axis.
        :type max: float.
        :param res: The resolution of the desired axis (centers) array.
        :type res: float.

        """
        # Axis properties.
        self.name = name
        self.min  = float(min)
        self.max  = float(max)
        self.res  = float(res)
        
        # Cell properties (positions on the axis).
        self.mid_points = _get_axis_mids(self.res, self.min, self.max)
        self.min_bounds = self.mid_points - self.res*0.5
        self.max_bounds = self.mid_points + self.res*0.5        
        
        # More axis properties.
        self.mid  = (self.min + self.max)*0.5
        self.size = self.mid_points.size

        
    def __shift__(self, shift: float):
        """Shift the axis by the input float (positive or negative)."""
        self.midpoint   = self.midpoint + float(shift)
        self.mid_points = self.mid_points + float(shift)
        self.min_bounds = self.min_bounds + float(shift)
        self.max_bounds = self.max_bounds + float(shift)
    
    
    def __on__(self, value: float):
        """Tests if input value exists on the axis (within min, max)."""
        return((self.min < value) & (value < self.max))
    
    
    def __index__(self, value: float):
        """Returns index of cell that contains input, if any, else None."""
        ix = np.where(np.logical_and(
            self.min_bounds.__le__(float(value)),     # Value less than x.
            self.max_bounds.__gt__(float(value))))[0] # Value greater than x.
        ix = None if len(ix) is 0 else ix[0]
        return(ix)
    
    
res, min_, max_  = (1.2, 1, 11)
ax = Axis("x", res, min_, max_)
ax.__dict__

### Tests

In [ ]:
# Tests: _get_axis_mids ------------------------------------------------------

res, min_, max_  = .153453, 1.15, 10.656456
midpoints_ = _get_axis_mids(res, min_, max_)
print("_get_axis_mids_t1:\t%s" % (midpoints_.min() < min_))
print("_get_axis_mids_t2:\t%s" % (midpoints_.max() > max_))

# Tests: Axis instantiation --------------------------------------------------

ax = Axis(name="x", res=1, min=1, max=10)
#print(ax.__dict__)

# Tests: Axis.__on__
Axis__on__t1 = ax.__on__(11.)
print("Axis.__on__t1:\t\t%s" % (Axis__on__t1 == False))
Axis__on__t2 = ax.__on__(2.)
print("Axis.__on__t2:\t\t%s" % (Axis__on__t2 == True))

# Tests: Axis.__index__
Axis__index__t1 = ax.__index__(5.5)
print("Axis.__index__t1:\t%s" % (Axis__index__t1 == 5))
Axis__index__t2 = ax.__index__(-0.1)
print("Axis.__index__t2:\t%s" % (Axis__index__t2 == None))

## class `Grid`

A `Grid` is made of one or more `Axis` instances.

In [ ]:
def _get_permuted_arrays(arrays):
    """Generate an array of voxel centroids from three dimensional arrays."""  
    arrays = [np.asarray(x) for x in arrays]  # Ensure all inputs are arrays.
    shape  = (len(x) for x in arrays)         # Get shape of each as a tuple.    
    ndims  = np.indices(shape)                # Get shape as array.
    index  = ndims.reshape(len(arrays), -1).T # Transpose.
    
    # Create empty 3d array, add axis positions, return.
    voxels = np.empty_like(index, dtype=arrays[0].dtype)
    for n, arr in enumerate(arrays):
        voxels[:, n] = arrays[n][index[:, n]]    
    return(voxels)


class Grid:
    """Initializes a grid for two or more axes."""
    
    def __init__(self, x=None, y=None, z=None):
        self.x, self.y, self.z = x, y, z     # Initialize input axes.
        self.__grid__()                      # Permute axes arrays.           

    def __grid__(self):
        """Wrapper for _get_permuted_arrays."""
        self.grid = _get_permuted_arrays((
            self.x.mid_points, 
            self.y.mid_points, 
            self.z.mid_points))
        
    def __add__(self, axis):
        """Add new Axis instance to grid."""
        setattr(self, axis.name, axis)

### Tests

In [ ]:
xaxis  = Axis("x", 0.5, 0.0, 10.0)
yaxis  = Axis("y", 0.5, 0.0, 10.0)
zaxis  = Axis("z", 0.5, 0.0, 10.0)
voxels = VoxelGrid(xaxis, yaxis, zaxis)

# Check the shape.
print(voxels.grid.shape == (xaxis.size * yaxis.size * zaxis.size, 3))

## class `Cloud`

```python
def _AxisSelect(array, r=0.02):
    """This doesn't work."""
    def _VoxelAxisSelect(voxel_center_value):
        """Vectorize point selection for input voxel.
        """
        # Get offset limits from voxel centroid.
        minOffset = voxel_center_value - r*0.5
        maxOffset = voxel_center_value + r*0.5
        # ValueSelect. ---------------------------------------------------
        # Return bool indicating if value inside reference offsets (voxel).
        def _ValueSelect(value):
            return(all([minOffset < value, value <= maxOffset]))
        # Vectorize the function to apply over array.
        ValueSelect = np.vectorize(_ValueSelect)
        # Evaluate over input axis.
        return(ValueSelect(array))
    # Vectorize the function to apply over array.
    return(np.vectorize(_VoxelAxisSelect))
```

```
----------------------------------------------------------------------
TRANSFORMATION MATRIX REFRESHER:

* Column 3, rows 1,2,3 defines the translation -------->       |
* The rotation matrix is the first three columns/rows:   +---------+    
      | 1 0 0 | -->                                      | 1 0 0 x | 
      | 0 1 0 | -->                                      | 0 1 0 y |
      | 0 0 1 | -->                                      | 0 0 1 z |
* We don't use last row (and must stay | 0 0 0 1 |) ---> | 0 0 0 1 |
* This matrix rotates 45° (pi/4) on Z axis:              +---------+                   
      | cos(θ) -sin(θ) 0.0 |
  R = | sin(θ)  cos(θ) 0.0 |
      | 0.0     0.0    1.0 |
      
----------------------------------------------------------------------
```

In [ ]:
import xarray as xr
import dask
import dask.dataframe as dd


def _read_dask_df(file: str, options: dict={sep=" ", names=None}):
    """Parse XYZ cloud to dask data frame."""
    try:
        return dd.read_csv(file, **options)
    except Exception as e:
        print("Error reading input cloud: %s" % file); raise(e); 


def _merge_dask_df(df_to_merge, merge_to_df):
    """Merge (by appending) df_to_merge into (existing) merge_to_df."""  
    
    # If existing df (merge_to_df) is None, return new df (df_to_merge).
    if merge_to_df is None:    
        return df_to_merge
    else:
        
        # Attempt to merge df_to_merge into merge_to_df. 
        try:                   
            df = dd.concat([merge_to_df, df_to_merge], axis=0)
            print("Successfully merged cloud.")
        
        # If failure, return existing df (merge_to_df).
        except Exception as e:                      
            print("Error merging cloud."); print(e)
            df = merge_to_df
        
        # Finally, if no exceptions, return merged df.
        finally:               
            return df          

    
class Cloud:
    """ """   
    
    ### Initialize empty cloud as a dask dataframe. --------------------------
    
    cloud     = None
    subclouds = {}
    computed  = {"x": False, "y": False, "z": False}
    
    ### Defaults: 3D scale and transformation matrix. ------------------------
    
    scale =  [1.0, 1.0, 1.0]    # Cloud scale       (x=1, y=1, z=1).
    trans =  [0.0, 0.0, 0.0]    # Cloud translation (None).
    rotat = [[1.0, 0.0, 0.0],   # Cloud rotation    (None).
             [0.0, 1.0, 0.0],
             [0.0, 0.0, 1.0]]
    
    
    ### Class instantiation. -------------------------------------------------
    
    def __init__(self, files: list=None, transform: list=None):
        """Takes as input a list of paths to point clouds."""
        # Add logic to -->>     # Update the transformation matrix.
        for f in files:         # Iterate over input point cloud files.
            self.__add__(f)     # Add cloud to class.
    
    
    ### Class methods: transformation. ---------------------------------------
    
    def __index__(self, axis: str):
        """Returns the index of the axis name."""
        return {"x": 0, "y": 1, "z": 2}[axis]
            
    def __matrix__(self, axis=None):
        """Returns transformation array for an axis or whole matrix."""    
        if axis is None:                      # If no axis given,
            return self.rotat + [self.trans]  # Merge and return matrix.
        else:                                 # Else if axis given,
            axix   = self.__index__(axis)     # Get the axis number.
            rotat = self.rotat[axix]          # Get the rotation matrix.
            return rotat + [self.trans[axix]] # Append the translation value.                              

    def __scale__(self, axis: str, value: float):
        """Scale point cloud along input axis."""
        axix = self.__index__(axis)           # Get the axis number
        scale[axix] = value                   # Replace scale matrix value.
        print("...")                          # Translate cloud.       
        
    def __translate__(self, axis: str, value: float):
        """Positive or negative shift (translation) along input axis."""
        axix = self.__index__(axis)           # Get the axis number
        trans[axix] = value                   # Replace translate matrix val.
        print("...")                          # Translate cloud.
        
    def __rotate__(self, axis: str, matrix: list):
        """Rotate point cloud along input axis."""        
        axix = self.__index__(axis)           # Get the axis number.
        rotat[axix] = matrix                  # Replace rotation matrix row.
        print("...")                          # Rotate cloud.    

        
    ### Class methods: point clouds. -----------------------------------------
            
    def __add__(self, file: str, merge: bool=False):
        """Parse XYZ cloud, apply transform, and append to class cloud."""
        self.subclouds[file] = _read_dask_df(file)
        if merge:
            self.cloud = _merge_dask_df(self.subclouds[file], self.cloud)

    def __compute__(self, axis: str=None):
        """Compute one or all axes in the cloud."""
        if axis is None:
            self.computed = {"x":True,"y":True,"z":True}  # All to computed.
            self.cloud = self.cloud.compute()             # Proc whole cloud.
        else:
            self.computed[axis] = True                    # Axis to computed.
            self.cloud[axis] = self.cloud[axis].compute() # Process one axis.

**Initialize with one cloud:**

In [ ]:
cloud1 = Cloud(["tls1.xyz"])
cloud1.cloud

## Next?

## MODULE `ptcloud`

In [ ]:
%%file ptcloud.py
#!/usr/bin/env python3
'''ptcloud'''

import sys     
import decimal
import numpy as np

# disallow scientific notation when printing
np.set_printoptions(suppress=True)


##############################################################################
# functions and classes.
##############################################################################


def _get_precision(x: float):
    return abs(decimal.Decimal(str(x)).as_tuple().exponent)


def _safe_arange(start, stop, step):
    return step * np.arange(start / step, stop / step)


def _get_axis_mids(res: float, min: float, max: float):
    """Get centers of cells on an input axis defined by its min, max, res.
    
    This necessarily convoluted routine ensures that all discrete points in a 
    point cloud will all fall inside the returned axis (cell centers) with
    the specified resolution.
    
    :param min: The minimum of all point positions along this axis.
    :type min: float.
    :param max: The maximum of all point positions along this axis.
    :type max: float.
    :param res: The resolution of the desired axis (centers) array.
    :type res: float.
    :returns:  numpy.array.
    
    """
    midpoint       = (max + min)*0.5
    sequence_left  = _safe_arange(midpoint - res, min - res, -res)
    sequence_right = _safe_arange(midpoint, max + res, res)
    midpoints      = np.concatenate([np.sort(sequence_left), sequence_right])
    return midpoints


class Axis:
    """The arrays and attributes that fully describe a cartesian axis.
    
    .. note::
        All attributes WITHOUT underscores describe axis properties.
        All attributes WITH underscores describe cell properties.    
    
    """
    
    def __init__(self, name: str, res: float, min: float, max: float):
        """Initializes an `Axis` class object.
        
        :param name: The name of this axis. Best to use one of [x, y, z].
        :type name: str.       
        :param min: The minimum of all point positions along this axis.
        :type min: float.
        :param max: The maximum of all point positions along this axis.
        :type max: float.
        :param res: The resolution of the desired axis (centers) array.
        :type res: float.

        """
        # Axis properties.
        self.name = name
        self.min  = float(min)
        self.max  = float(max)
        self.res  = float(res)
        
        # Cell properties (positions on the axis).
        self.mid_points = _get_axis_mids(self.res, self.min, self.max)
        self.min_bounds = self.mid_points - self.res*0.5
        self.max_bounds = self.mid_points + self.res*0.5        
        
        # More axis properties.
        self.mid  = (self.min + self.max)*0.5
        self.size = self.mid_points.size

        
    def __shift__(self, shift: float):
        """Shift the axis by the input float (positive or negative)."""
        self.midpoint   = self.midpoint + float(shift)
        self.mid_points = self.mid_points + float(shift)
        self.min_bounds = self.min_bounds + float(shift)
        self.max_bounds = self.max_bounds + float(shift)
    
    
    def __on__(self, value: float):
        """Tests if input value exists on the axis (within min, max)."""
        return((self.min < value) & (value < self.max))
    
    
    def __index__(self, value: float):
        """Returns index of cell that contains input, if any, else None."""
        ix = np.where(np.logical_and(
            self.min_bounds.__le__(float(value)),     # Value less than x.
            self.max_bounds.__gt__(float(value))))[0] # Value greater than x.
        ix = None if len(ix) is 0 else ix[0]
        return(ix)
    

##############################################################################
# test.
############################################################################## 
    
    
def test(res=.5, min_=0., max_=10.):
    """ """
    
    # Tests _get_axis_mids ---------------------------------------------------
    midpoints_ = _get_axis_mids(res, min_, max_)
    print("_get_axis_mids_t1:\t%s" % (midpoints_.min() < min_))
    print("_get_axis_mids_t2:\t%s" % (midpoints_.max() > max_))

    # Tests: Axis instantiation ----------------------------------------------
    ax = Axis(name="x", res=res, min=min_, max=max_)
    # Tests: Axis.__on__
    Axis__on__t1 = ax.__on__(11.)
    print("Axis.__on__t1:\t\t%s" % (Axis__on__t1 == False))
    Axis__on__t2 = ax.__on__(2.)
    print("Axis.__on__t2:\t\t%s" % (Axis__on__t2 == True))
    # Tests: Axis.__index__
    Axis__index__t1 = ax.__index__(5.5)
    print("Axis.__index__t1:\t%s" % (Axis__index__t1 == 5))
    Axis__index__t2 = ax.__index__(-0.1)
    print("Axis.__index__t2:\t%s" % (Axis__index__t2 == None))

    # Test VoxelGrid instantiation. ------------------------------------------
    xaxis  = Axis("x", res, min_, max_)
    yaxis  = Axis("y", res, min_, max_)
    zaxis  = Axis("z", res, min_, max_)
    voxels = VoxelGrid(xaxis, yaxis, zaxis)
    # Check the shape.
    print(voxels.grid.shape == (xaxis.size * yaxis.size * zaxis.size, 3))
    
    
##############################################################################
# main.
##############################################################################


def main(filename): 
    """main."""
    print("...")


##############################################################################
# run.
##############################################################################


if __name__ == "__main__":

    
    # Try to get arguments.
    try:
        script, xyz, offset = sys.argv
    except:
        sys.exit("Takes two args: <lidar file>.xyz <resolution>")
    
    main()
        
    # Resolution of grid.
    #xvoxel, yvoxel, zvoxel = main(xyz, int(offset))

## Execute `ptcloud.py`

In [ ]:
%run -i ptcloud.py tls1.xyz 1

!echo Print first three of each voxel axis as a dict:
dict(x=xvoxel[:3], y=yvoxel[:3], z=zvoxel[:3])

## Import `ptcloud.py

In [ ]:
import pandas as pd
from ptcloud import GetAxisCentroids, GetVoxels


    

class Point:
    
    def __init__(self, x, y):
        self.x, self.y = x, y
        
    def __str__(self):
        return "{}, {}".format(self.x, self.y)
    
    def __neg__(self):
        return Point(-self.x, -self.y)

    def __add__(self, point):
        return Point(self.x+point.x, self.y+point.y)
    
    def __sub__(self, point):
        return self + -point 
    

def main():
    """ """
    
    # Read tls 1 and 2 with pandas.
    df = pd.read_csv(
        filename, 
        sep=" ", 
        skiprows=1, 
        index_col=False, 
        names=["x","y","z","r","g","b"])
    
    # Arrays for each axis of the cloud.
    x = df.x.to_numpy()
    y = df.y.to_numpy()
    z = df.z.to_numpy()
    
    # Get bounds of each axis.
    xbnds = [min(x), max(x)]
    ybnds = [min(y), max(y)]
    zbnds = [min(z), max(z)]
    
    # Get center of each axis.
    xcent = sum(xbnds)*0.5
    ycent = sum(ybnds)*0.5
    zcent = sum(zbnds)*0.5
    
    # Get arrays of centroids for each axis.
    xmids = GetAxisCentroids(*xbnds, offset)
    ymids = GetAxisCentroids(*ybnds, offset)
    zmids = GetAxisCentroids(*zbnds, offset)
    
    # Get permuted array of voxel centroids. 
    voxels = GetVoxels(tuple([xmids, ymids, zmids]))
    xvoxel, yvoxel, zvoxel = voxels[0], voxels[1], voxels[2]
    
    return(xvoxel, yvoxel, zvoxel)

## Dev `ptcloud.py`

*Some type of decorated function for selecting voxels:*
```python
def _AxisSelect(array, r=0.02):
    """This doesn't work."""
    def _VoxelAxisSelect(voxel_center_value):
        """Vectorize point selection for input voxel.
        """
        # Get offset limits from voxel centroid.
        minOffset = voxel_center_value - r*0.5
        maxOffset = voxel_center_value + r*0.5
        # ValueSelect. ---------------------------------------------------
        # Return bool indicating if value inside reference offsets (voxel).
        def _ValueSelect(value):
            return(all([minOffset < value, value <= maxOffset]))
        # Vectorize the function to apply over array.
        ValueSelect = np.vectorize(_ValueSelect)
        # Evaluate over input axis.
        return(ValueSelect(array))
    # Vectorize the function to apply over array.
    return(np.vectorize(_VoxelAxisSelect))
```